# Project 1 - Team BAK

## Step 1 - Getting started

In [34]:
#Import some libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
from helpfulfun import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
#load data
import csv
def readCSVToNumpyArray(dataset):
    values = [[]]
    with open(dataset) as f:
        counter = 0
        for i in csv.reader(f):
            for j in i:
                try:
                    values[counter].append(float(j))
                except ValueError:
                    values[counter].append(j)
            counter = counter + 1
            values.append([])

    data = np.array(values[:-1],dtype='object')

    return data

train = readCSVToNumpyArray('train.csv')[1:,]
test = readCSVToNumpyArray('test.csv')[1:,]
x_tr = train[:,2:]
y_tr = train[:,1]
x_te = test[:,2:]

train[:,0] = [int(x) for x in train[:,0]]
test[:,0] = [int(x) for x in test[:,0]]

x_tr.shape, y_tr.shape, x_te.shape

((250000, 30), (250000,), (568238, 30))

In [36]:
# import the datasets
#train_list = np.genfromtxt("train.csv", dtype=None, delimiter=",", skip_header =1, unpack=True, encoding=None)
#train = np.array(train_list)
#test_list = np.genfromtxt("test.csv", dtype=None, delimiter=",", skip_header =1, unpack=True, encoding=None)
#test = np.array(test_list)

In [37]:
#x_te = test[2:].T

In [38]:
#y_tr = train[1]
#x_tr = train[2:].T
#print("x: ", x_tr.shape, " y: ", y_tr.shape)

## Step 2 - Preprocessing

In [39]:
y_tr = np.where(y_tr == "s",1,0)
print(y_tr.shape, " and y[:5]: ", y_tr[:5])

(250000,)  and y[:5]:  [1 0 0 0 0]


In [40]:
# delete features with more than 30% NaN values
x_tr_prep = np.delete(x_tr, 0, 1) # infer this one later
x_te_prep = np.delete(x_te, 0, 1) # infer this one later
for i in [4, 4, 4, 9, 18, 18, 18, 18, 18, 18, 18]:
    x_tr_prep = np.delete(x_tr_prep, i, 1)
    x_te_prep = np.delete(x_te_prep, i, 1)
x_tr_prep = x_tr_prep.astype(float)
x_te_prep = x_te_prep.astype(float)
x_tr_prep.shape

(250000, 18)

In [41]:
x, mean_x, std_x = standardize(x_tr_prep)
x_te, _, _ = standardize(x_te_prep)

In [42]:
x.shape, x_te.shape

((250000, 19), (568238, 19))

## Step 3 - Implement ML Methods

In [43]:
initial_w = np.zeros((x.shape[1],), dtype=float)

In [44]:
def get_predictions(x, best_w):
    preds = x.dot(best_w)
    y_te = np.where(preds < .5,-1,1)
    y_pred = np.c_[test[:, 0], y_te]
    print(y_pred[0:5])
    y_pred = np.insert(y_pred, 0, ["Id", "Prediction"], axis=0)
    return y_pred

In [45]:
test[:,1].shape

(568238,)

#### Linear regression using gradient descent

In [46]:
def mean_squared_error_gd(y, tx, initial_w, max_iters, gamma):
    """The Gradient Descent (GD) algorithm.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of GD
        gamma: a scalar denoting the stepsize
        
    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of GD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of GD 
    """
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        gradient = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        
        w = w - gamma * gradient
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("GD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws
    

In [47]:
losses, ws = mean_squared_error_gd(y_tr, x, initial_w, max_iters=150, gamma=.005)

GD iter. 0/149: loss=0.171334, w0=0.0017133399999999998, w1=0.00043645910164310674
GD iter. 1/149: loss=0.1649925688549367, w0=0.00332096214587879, w1=0.0008386390404597022
GD iter. 2/149: loss=0.1596486017372754, w0=0.0048317425994134165, w1=0.00120943363357687
GD iter. 3/149: loss=0.15513735265013423, w0=0.006253789666325029, w1=0.0015514862427284718
GD iter. 4/149: loss=0.15132140940056046, w0=0.007594510397486284, w1=0.0018672114987907896
GD iter. 5/149: loss=0.14808615665186411, w0=0.008860671412039476, w1=0.002158815141566262
GD iter. 6/149: loss=0.14533599206028733, w0=0.010058454443722461, w1=0.0024283121383331563
GD iter. 7/149: loss=0.14299117049171767, w0=0.0111935070682075, w1=0.0026775432304924586
GD iter. 8/149: loss=0.14098517206359573, w0=0.012270989029539839, w1=0.0029081900446875342
GD iter. 9/149: loss=0.139262507062407, w0=0.013295614547490212, w1=0.003121788892940387
GD iter. 10/149: loss=0.13777688521990067, w0=0.014271690954509957, w1=0.0033197433755431014
GD ite

GD iter. 118/149: loss=0.11496527230522288, w0=0.05760943404587481, w1=0.0053295449302295135
GD iter. 119/149: loss=0.11490231855593183, w0=0.057853129889593014, w1=0.005303023072507151
GD iter. 120/149: loss=0.11484027949595559, w0=0.05809530358394553, w1=0.005276091891138473
GD iter. 121/149: loss=0.11477914139630109, w0=0.05833596673072132, w1=0.005248754562743506
GD iter. 122/149: loss=0.11471889073411963, w0=0.05857513082998631, w1=0.0052210142353379735
GD iter. 123/149: loss=0.1146595141896097, w0=0.058812807282090755, w1=0.005192874028953586
GD iter. 124/149: loss=0.11460099864296637, w0=0.05904900738955381, w1=0.005164337036218541
GD iter. 125/149: loss=0.11454333117137747, w0=0.05928374235883561, w1=0.005135406322901581
GD iter. 126/149: loss=0.1144864990460642, w0=0.05951702330200617, w1=0.005106084928422669
GD iter. 127/149: loss=0.11443048972936727, w0=0.05974886123831979, w1=0.005076375866333065
GD iter. 128/149: loss=0.1143752908718762, w0=0.05997926709570261, w1=0.005046

In [48]:
best_ws = ws[-1]
pred = get_predictions(x_te, best_ws)

[[350000 -1]
 [350001 -1]
 [350002 -1]
 [350003 -1]
 [350004 1]]


In [17]:
np.savetxt("submission.csv", pred, fmt="%s", delimiter=",")

#### Linear regression using stochastic gradient descent


In [18]:
def mean_squared_error_sgd(y, tx, initial_w, max_iters, gamma):
    """The Stochastic Gradient Descent algorithm (SGD).
            
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        batch_size: a scalar denoting the number of data points in a mini-batch used for computing the stochastic gradient
        max_iters: a scalar denoting the total number of iterations of SGD
        gamma: a scalar denoting the stepsize
        
    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of SGD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of SGD 
    """
    
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    
    for n_iter in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
            gradient = compute_stoch_gradient(minibatch_y, minibatch_tx, w)
            loss = compute_loss(y, tx, w)
            w = w - gamma * gradient
        # store w and loss
            ws.append(w)
            losses.append(loss)

        print("SGD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws
    

#### Least squares regression using normal equations

In [19]:
def least_squares(y, tx):
    opt_weights = np.linalg.solve(tx.T.dot(tx)).dot(tx.T.dot(y))
    e = y - tx.dot(opt_weights)
    mse = 1/(2*len(y)) * e.T.dot(e)
    return opt_weights, mse

#### Ridge regression using normal equations. Given the large number of parameters (that may be correlated), it's better to use ridge regression or lasso.

In [20]:
def ridge_regression(y, tx, lambda_ ):
    aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

#### Logistic regression using gradient descent or SGD (y ∈ {0, 1})

In [60]:
# calculating loss with sigmoid and using gradient descent
def logistic_regression_gradient_descent(y_tr, x):
    max_iter = 500
    threshold = 1e-8
    gamma = .5
    losses = []

    # build tx
    #tx = np.c_[np.ones((y_tr.shape[0],)), x]
    tx = x
    initial_w = np.zeros((tx.shape[1],), dtype=float)
    w = initial_w

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y_tr, tx, w, gamma)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_gradient_descent", True)
    print("loss={l}".format(l=calculate_loss_lr(y_tr, tx, w)))
    
    return losses[-1], w

In [61]:
tx = np.c_[np.ones((y_tr.shape[0],)), x]
initial_w = np.zeros((tx.shape[1],1), dtype=float)
w = initial_w
w.shape, tx.shape

((20, 1), (250000, 20))

In [62]:
losses_logreg, ws_logreg = logistic_regression_gradient_descent(y_tr, x)

Current iteration=0, loss=0.6931471805599171
Current iteration=100, loss=0.6113890294856259
Current iteration=200, loss=0.5993193152531957
Current iteration=300, loss=0.5912192725832195
Current iteration=400, loss=0.5852477360620909
loss=0.5806758546942088


In [63]:
def get_predictions_logistic(x, best_w):
    preds = sigmoid(x.dot(best_w).reshape((x.shape[0],1)))
    y_te = np.where(preds < .5,-1,1)
    y_pred = np.c_[test[:, 0], y_te]
    print(y_pred[0:5])
    y_pred = np.insert(y_pred, 0, ["Id", "Prediction"], axis=0)
    return y_pred

In [64]:
best_ws = ws_logreg
best_ws.shape
#pred = get_predictions_logistic(x_te, best_ws)

(19,)

In [66]:
pred = get_predictions_logistic(x = x_te, best_w = best_ws)
np.savetxt("submission2.csv", pred, fmt="%s", delimiter=",")

[[350000 -1]
 [350001 -1]
 [350002 -1]
 [350003 -1]
 [350004 -1]]


In [22]:
def logistic_regression_penalized_gradient_descent(y, x):
    # init parameters
    max_iter = 2000
    gamma = 0.5
    lambda_ = 0.0005
    threshold = 1e-8
    losses = []

    # build tx
    tx = np.c_[np.ones((y_tr.shape[0],)), x]
    initial_w = np.zeros((tx.shape[1],), dtype=float)
    w = initial_w

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_penalized_gradient_descent", True)
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    return losses[-1], w
    

In [ ]:
losses_p, ws_p = logistic_regression_penalized_gradient_descent(y_tr, x)

In [ ]:
def logistic_regression_newton_method(y, x):
    # init parameters
    max_iter = 100
    threshold = 1e-8
    lambda_ = 0.1
    gamma = 1.
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], )), x]
    w = np.zeros((tx.shape[1], 1))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, tx, w, gamma)
        # log info
        if iter % 1 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))

        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_newton_method", True)
    print("loss={l}".format(l=calculate_loss(y, tx, w)))
    
    return losses, w
    

In [ ]:
losses_nm, ws_nm = logistic_regression_newton_method(y_tr, x)